In [1]:
# --- QUESTION 5: Re-Train & Interpretability Setup ---

# Install Libraries
print("Installing Captum & Utilities...")
!pip install captum torchtext==0.17.0 transformers==4.38.2 datasets scikit-learn portalocker > /dev/null

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import GloVe
from datasets import load_dataset
from captum.attr import LayerIntegratedGradients, visualization as viz
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Setup Complete. Device: {device}")

# Data Pipeline (IMDb + GloVe)
print("Loading Data & GloVe...")
dataset = load_dataset("imdb")
train_data_list = list(dataset['train'])
test_data_list = list(dataset['test'])

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for item in data_iter:
        yield tokenizer(item['text'])

vocab = build_vocab_from_iterator(yield_tokens(train_data_list), specials=["<unk>", "<pad>"], min_freq=5)
vocab.set_default_index(vocab["<unk>"])

glove = GloVe(name='6B', dim=300)
embedding_matrix = torch.zeros((len(vocab), 300))
for i, token in enumerate(vocab.get_itos()):
    embedding_matrix[i] = glove.get_vecs_by_tokens(token)

def collate_batch(batch):
    labels = []
    texts = []
    for item in batch:
        labels.append(item['label'])
        tokens = tokenizer(item['text'])
        numerical_tokens = [vocab[token] for token in tokens]
        texts.append(torch.tensor(numerical_tokens, dtype=torch.long))
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab["<pad>"])
    labels = torch.tensor(labels, dtype=torch.float)
    return texts_padded.to(device), labels.to(device)

train_loader = DataLoader(train_data_list, batch_size=64, shuffle=True, collate_fn=collate_batch)

# Model Definition (Bi-GRU)
class RecurrentClassifier(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim, num_layers, dropout):
        super().__init__()
        _, embed_dim = embedding_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False, padding_idx=vocab["<pad>"])
        self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers, bidirectional=True, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, 1)

    def forward(self, text):
        embedded = self.embedding(text)
        _, hidden = self.rnn(embedded)
        final_hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        return self.fc(self.dropout(final_hidden))

print("Training Bi-GRU Model (5 Epochs)...")
model = RecurrentClassifier(embedding_matrix, 64, 1, 0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

model.train()
for epoch in range(5):
    start = time.time()
    epoch_loss = 0
    for text, labels in train_loader:
        optimizer.zero_grad()
        preds = model(text).squeeze(1)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {epoch_loss/len(train_loader):.4f} | Time: {time.time()-start:.0f}s")

print(" Model Re-Trained and Ready for Analysis!")

Installing Captum & Utilities...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
torchtune 0.6.1 requires torchdata==0.11.0, but you have torchdata 0.7.1 which is incompatible.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
tor

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                               
100%|█████████▉| 399999/400000 [00:44<00:00, 9013.85it/s]


Training Bi-GRU Model (5 Epochs)...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Epoch 1: Loss = 0.5290 | Time: 18s
Epoch 2: Loss = 0.2256 | Time: 17s
Epoch 3: Loss = 0.1042 | Time: 17s
Epoch 4: Loss = 0.0500 | Time: 18s
Epoch 5: Loss = 0.0238 | Time: 18s
 Model Re-Trained and Ready for Analysis!


In [2]:
# --- STEP 2 : Interpretability with Integrated Gradients ---
from captum.attr import LayerIntegratedGradients, visualization as viz
import torch.backends.cudnn as cudnn # Import cudnn backend

# Define Wrapper for Captum
# We output raw logits (scores) instead of probabilities to avoid saturation
def forward_logits(input, target_class=None):
    return model(input)

# Initialize Layer Integrated Gradients
lig = LayerIntegratedGradients(forward_logits, model.embedding)

# Interpretation Function
def interpret_sentence(sentence):
    model.eval()
    
    # Tokenize
    tokenized = [vocab[token] for token in tokenizer(sentence)]
    input_tensor = torch.tensor(tokenized, dtype=torch.long).unsqueeze(0).to(device)
    
    # Generate Prediction (Just for display info)
    with torch.no_grad():
        logits = model(input_tensor)
        prob = torch.sigmoid(logits).item()
        pred_label = 1 if prob > 0.5 else 0
    
    # Calculate Attributions
    with cudnn.flags(enabled=False):
        attributions, delta = lig.attribute(
            input_tensor,
            target=0, 
            n_steps=50,
            return_convergence_delta=True
        )
    
    # Prepare Data for Visualization
    return viz.VisualizationDataRecord(
        word_attributions=attributions.sum(dim=2).squeeze(0),
        pred_prob=prob,
        pred_class=str(pred_label),
        true_class="?", 
        attr_class=str(pred_label),
        attr_score=attributions.sum(),
        raw_input_ids=tokenizer(sentence),
        convergence_score=delta
    )

# 4. Run Analysis
print("--- Model Interpretability Analysis ---")
print("Visualizing word importance (Red=Negative, Green=Positive)\n")

test_sentences = [
    "This movie was absolutely fantastic and I loved every moment of it.", 
    "The plot was boring and the acting was terrible.", 
    "I thought it would be good but it turned out to be a waste of time.", 
    "Not the best movie ever, but certainly worth watching.", 
    "The cinematography was great, but the story made no sense." 
]

vis_records = []
for text in test_sentences:
    print(f"Processing: {text}")
    vis_records.append(interpret_sentence(text))

# Render Visualization
print("\n--- Visualization Results ---")
_ = viz.visualize_text(vis_records)

--- Model Interpretability Analysis ---
Visualizing word importance (Red=Negative, Green=Positive)

Processing: This movie was absolutely fantastic and I loved every moment of it.
Processing: The plot was boring and the acting was terrible.
Processing: I thought it would be good but it turned out to be a waste of time.
Processing: Not the best movie ever, but certainly worth watching.
Processing: The cinematography was great, but the story made no sense.

--- Visualization Results ---


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
?,1 (0.99),1,8.44,this movie was absolutely fantastic and i loved every moment of it .
?,0 (0.00),0,-3.16,the plot was boring and the acting was terrible .
?,0 (0.02),0,0.72,i thought it would be good but it turned out to be a waste of time .
?,1 (0.84),1,5.15,"not the best movie ever , but certainly worth watching ."
?,0 (0.41),0,3.34,"the cinematography was great , but the story made no sense ."


In [3]:
# --- STEP 3 : Failure Analysis & Uncertainty ---
import torch.distributions as dist
import numpy as np
from captum.attr import LayerIntegratedGradients, visualization as viz
import torch.backends.cudnn as cudnn

print("---  Task 5c: Hunting for Failures ---")

# Find Failure Cases
failure_cases = []
model.eval()

print("Scanning test set for mistakes...", end="")
with torch.no_grad():
    for i, item in enumerate(test_data_list):
        text = item['text']
        label = item['label']
        
        # Prepare input
        tokenized = [vocab[token] for token in tokenizer(text)]
        input_tensor = torch.tensor(tokenized, dtype=torch.long).unsqueeze(0).to(device)
        
        # Predict
        logits = model(input_tensor)
        prob = torch.sigmoid(logits).item()
        pred = 1 if prob > 0.5 else 0
        
        # Compare 
        if pred != label:
            # Store max 5 cases
            failure_cases.append((text, label, prob))
            print("!", end="")
        
        if len(failure_cases) >= 5:
            break
print(" Done!")

# Visualize the Failures
print("\n--- Visualizing 5 Failure Cases ---")
vis_records_fail = []

for text, true_label, prob in failure_cases:
    pred_label = 1 if prob > 0.5 else 0
    print(f"Analyzing Failure: True={true_label}, Pred={pred_label} (Conf: {prob:.2f})")
    
    tokenized = [vocab[token] for token in tokenizer(text)]
    input_tensor = torch.tensor(tokenized, dtype=torch.long).unsqueeze(0).to(device)
    
    # Calculate attribution (Disable CuDNN for RNN backward)
    with cudnn.flags(enabled=False):
        attributions, delta = lig.attribute(
            input_tensor, target=0, n_steps=50, return_convergence_delta=True
        )
    
    rec = viz.VisualizationDataRecord(
        word_attributions=attributions.sum(dim=2).squeeze(0),
        pred_prob=prob,
        pred_class=str(pred_label),
        true_class=str(true_label),
        attr_class=str(1 if prob>0.5 else 0), 
        attr_score=attributions.sum(),
        raw_input_ids=tokenizer(text),
        convergence_score=delta
    )
    vis_records_fail.append(rec)

_ = viz.visualize_text(vis_records_fail)


# Task 5d: Quantify Uncertainty (Entropy)
print("\n---  Task 5d: Model Uncertainty (Entropy) ---")

entropies = []
print("Calculating entropy over 1000 test samples...", end="")

with torch.no_grad():
    for i, item in enumerate(test_data_list[:1000]):
        text = item['text']
        tokenized = [vocab[token] for token in tokenizer(text)]
        input_tensor = torch.tensor(tokenized, dtype=torch.long).unsqueeze(0).to(device)
        
        logits = model(input_tensor)
        prob = torch.sigmoid(logits)
        
        # Binary Entropy
        p = prob.clamp(1e-6, 1 - 1e-6)
        entropy = -(p * torch.log(p) + (1 - p) * torch.log(1 - p))
        entropies.append(entropy.item())

avg_entropy = np.mean(entropies)
print(" Done!")

print(f"\n📊 Average Model Entropy: {avg_entropy:.4f}")
print("(Lower = More Confident, Higher = More Uncertain)")

---  Task 5c: Hunting for Failures ---
Scanning test set for mistakes...!!!!! Done!

--- Visualizing 5 Failure Cases ---
Analyzing Failure: True=0, Pred=1 (Conf: 0.97)
Analyzing Failure: True=0, Pred=1 (Conf: 0.97)
Analyzing Failure: True=0, Pred=1 (Conf: 0.57)
Analyzing Failure: True=0, Pred=1 (Conf: 1.00)
Analyzing Failure: True=0, Pred=1 (Conf: 0.97)



---  Task 5d: Model Uncertainty (Entropy) ---
Calculating entropy over 1000 test samples... Done!

📊 Average Model Entropy: 0.0987
(Lower = More Confident, Higher = More Uncertain)
